In [48]:
import pandas as pd
import mysql.connector

In [49]:
conn = mysql.connector.connect(
    host="localhost",
    user="root_user",
    password="root@123",
    database="healthcare_analytics"
)

print("MySQL connection successful")


MySQL connection successful


In [50]:
cursor = conn.cursor()

In [51]:
# Load CSV
df = pd.read_csv(r"C:\PROJECTS\Healthcare_Readmission_Analytics\dataset\hospital_readmissions_cleaned.csv")  # Make sure this file exists in your working directory

In [52]:
# Create table if not exists
cursor.execute("""
    CREATE TABLE IF NOT EXISTS patient_data (
        age FLOAT,
        time_in_hospital INT,
        n_lab_procedures INT,
        n_procedures INT,
        n_medications INT,
        n_outpatient INT,
        n_inpatient INT,
        n_emergency INT,
        medical_specialty VARCHAR(255),
        diag_1 VARCHAR(50),
        diag_2 VARCHAR(50),
        diag_3 VARCHAR(50),
        glucose_test VARCHAR(50),
        A1Ctest VARCHAR(50),
        change_med VARCHAR(50),
        diabetes_med VARCHAR(50),
        readmitted INT
    )
""")

In [53]:
insert_query = """
INSERT INTO patient_data (
    age, time_in_hospital, n_lab_procedures, n_procedures, n_medications,
    n_outpatient, n_inpatient, n_emergency, medical_specialty,
    diag_1, diag_2, diag_3, glucose_test, A1Ctest,
    change_med, diabetes_med, readmitted
) VALUES (
    %s, %s, %s, %s, %s,
    %s, %s, %s, %s,
    %s, %s, %s, %s, %s,
    %s, %s, %s
)
"""

data = [
    (
        float(row['age']),
        int(row['time_in_hospital']),
        int(row['n_lab_procedures']),
        int(row['n_procedures']),
        int(row['n_medications']),
        int(row['n_outpatient']),
        int(row['n_inpatient']),
        int(row['n_emergency']),
        row['medical_specialty'],
        row['diag_1'],
        row['diag_2'],
        row['diag_3'],
        row['glucose_test'],
        row['a1ctest'],
        row['change'],
        row['diabetes_med'],
        int(row['readmitted'])
    )
    for _, row in df.iterrows()
]

BATCH_SIZE = 500  # safe for MySQL

for i in range(0, len(data), BATCH_SIZE):
    batch = data[i:i + BATCH_SIZE]
    cursor.executemany(insert_query, batch)
    conn.commit()

print("Data inserted successfully in batches")




Data inserted successfully in batches
